In [1]:
from sqlalchemy.ext.asyncio import async_sessionmaker, create_async_engine
import sqlalchemy as sqla
from config import env


engine = create_async_engine(
    sqla.URL.create(
        host=env.DB_URL,
        drivername=env.DB_DRIVER,
        username="postgres",
        password="postgres",
        # port=env.DB_PORT,
        port=5432,
        database=env.DB_NAME,
    ),
    pool_pre_ping=True,
)

ss = async_sessionmaker(
    engine,
    expire_on_commit=False,
)()


In [2]:
from db.postgresql import init_db

await init_db()

In [2]:
from db.postgresql.models.product import (
    Product,
    ProductPriceHistory,
    ProductStockHistory,
)
from tqdm.autonotebook import tqdm
from datetime import datetime
import datetime as dt

/tmp/ipykernel_11315/2328573263.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
MAX_PRICE_CHANGE_AMOUNT = 30
MIN_PRICE_CHANGE_AMOUNT = 5

MAX_STOCK_CHANGE_AMOUNT = 30
MIN_STOCK_CHANGE_AMOUNT = 5

MAX_ITEM_PER_ORDER = 5
MIN_ITEM_PER_ORDER = 3

MAX_AMOUNT_OF_ITEM_IN_ORDER = 5
MIN_AMOUNT_OF_ITEM_IN_ORDER = 3

MAX_PRICE = 40
MIN_PRICE = 5

MAX_SALE = 30
MIN_SALE = 0

MAX_PRICE_CHANGE_INTERVAL_DAY = 70
MIN_PRICE_CHANGE_INTERVAL_DAY = 20


MAX_RESTOCK_INTERVAL = 70
MIN_RESTOCK_INTERVAL = 20

In [4]:
USER_AMOUNT = 100

STAFF_AMOUNT = 40
SHIPPER_AMOUNT = 100

MAX_ORDER_PER_USER = 10
MIN_ORDER_PER_USER = 2

MAX_ORDER_INTERVAL_DAY = 14
MIN_ORDER_INTERVAL_DAY = 2

In [5]:
import random
from passlib.context import CryptContext
import faker

from db.postgresql.models.user_account import UserAccount, UserAccountStatus

fake = faker.Faker()

In [6]:
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")


def generate_phone() -> str:
    num_arr = ["0"]

    for _ in range(9):
        num_arr.append(str(random.randint(0, 9)))

    return "".join(num_arr)


user_password = pwd_context.hash("password")

(trapped) error reading bcrypt version
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/passlib/handlers/bcrypt.py", line 620, in _load_backend_mixin
    version = _bcrypt.__about__.__version__
AttributeError: module 'bcrypt' has no attribute '__about__'


In [9]:
import uuid

pbar = tqdm(desc="Adding customer account", total=USER_AMOUNT)

async with ss.begin():
    for _ in range(USER_AMOUNT):
        username = fake.user_name()
        id = str(uuid.uuid4())

        staff = UserAccount(
            id=id,
            email=fake.email(),
            username=username,
            profile_name=username,
            password=user_password,
            token=username,
            status=random.choices(
                [UserAccountStatus.NORMAL, UserAccountStatus.BANNED],
                [0.9, 0.1],
            )[0],
            address=fake.address(),
            profile_pic_uri="defaultProfile",
            profile_description=" ",
            phone=generate_phone(),
        )

        pbar.update()

        ss.add(staff)

    await ss.flush()


Adding customer account:   0%|          | 0/100 [00:00<?, ?it/s]

In [13]:
import uuid

from db.postgresql.models.staff_account import (
    AccountStatus,
    AccountType,
    EmployeeInfo,
    StaffAccount,
)


async def generate_staff(type: AccountType, amount: int):
    pbar = tqdm(desc="Adding customer account", total=amount - 1)

    id_list = []
    async with ss.begin():
        for _ in range(amount):
            username = fake.user_name()
            id = str(uuid.uuid4())

            id_list.append(id)
            staff = StaffAccount(
                id=id,
                username=username,
                password=user_password,
                type=type,
                token=username,
                status=random.choices(
                    [
                        AccountStatus.ACTIVE,
                        AccountStatus.DISABLE,
                    ],
                    [0.9, 0.1],
                )[0],
            )

            info = EmployeeInfo(
                account_id=id,
                ssn=generate_phone(),
                phonenumber=generate_phone(),
                realname=fake.name(),
                email=fake.email(),
                dob=fake.date_of_birth(),
            )

            pbar.update()

            ss.add(staff)
            ss.add(info)

        await ss.flush()
    return id_list

In [12]:
await generate_staff(AccountType.STAFF, STAFF_AMOUNT)


Adding customer account:   0%|          | 0/39 [00:00<?, ?it/s]

In [ ]:
await generate_staff(AccountType.SHIPPER, STAFF_AMOUNT)

In [17]:
from db.postgresql.models.staff_account import StaffAccount, AccountType

async with ss.begin():
    product_amount = await ss.scalar(sqla.select(sqla.func.count(Product.id))) or 0
    product_ids = (await ss.scalars(sqla.select(Product.id))).all()
    account_ids = (await ss.scalars(sqla.select(UserAccount.id))).all()
    account_amount = (
        await ss.scalar(
            sqla.select(
                sqla.func.count(
                    UserAccount.id,
                )
            )
        )
        or 0
    )
    shippers = (
        await ss.scalars(
            sqla.select(StaffAccount.id).filter(
                StaffAccount.type == AccountType.SHIPPER
            )
        )
    ).all()

    staffs = (
        await ss.scalars(
            sqla.select(StaffAccount.id).filter(
                StaffAccount.type == AccountType.STAFF,
            )
        )
    ).all()

In [ ]:
from datetime import time, timedelta

from db.postgresql.models.shipper import ShipperAvailbility


async with ss.begin():
    for s in shippers:
        start_time = random.randint(1, 23)
        end_time = (start_time + random.randint(4, 7)) % 24

        start_ship = time(start_time)

        end_ship = time(end_time)

        sa = ShipperAvailbility(
            id=str(s),
            start_shift=start_ship,
            end_shift=end_ship,
            occupied=fake.boolean(20),
        )
        ss.add(sa)

    await ss.commit()

27990c37-676f-415c-8995-2e750904f208
59f41e0c-6c7d-4379-91d2-86d72c9115eb
3fcf721a-cc6c-40c0-8346-e05d9d4af7e4
8abc6ed4-b73f-4f55-9fa6-3bbd4cd3f3de
7dcab5c1-3421-467a-bee5-5c4d587f2026
8d32ba23-2caf-4485-8f23-4f952b500f7b
c102d7bc-053d-4012-9e73-22a9835cbd40
6334738e-7e12-418c-801a-6a344aa59bce
bfdea94d-316e-47bc-a6ff-641d3500a97e
e11b9238-d7e5-4f1c-a684-90571b64b707
8381dfcb-e485-4e48-9999-6e8cd133fd3e
6e04c883-89a4-4f25-8fac-87bd35903340
b09e00f0-5db9-4d61-9580-9a6afb7298d1
23fcc1e2-ee8c-4cc5-8e49-6bad8a25bff2
d95c2235-ab2c-4fbd-ae0e-e5fe497934f1
d60af480-9289-4e49-8484-0879786e9bc1
ef886c5d-b3df-47b2-b9d0-3ae6cd1d21de
19327e3f-18f9-448c-ab5d-fc38ff658da0
a4ab0b32-2dd7-49b4-aaff-ed6c1f513892
0bf64856-f6ba-4ac0-8c90-6a609017bef2
44f0d736-be79-49ab-81e8-68ff9bc2b67a
4f7aece1-d1fd-48fe-bf02-52a1bb43a6da
6856d200-dcaa-4713-974d-ecf177f55637
2f5afa1f-2644-4686-92d1-adbd8838aa21
2e108419-501f-4c54-814c-7feb3311e7c2
265f5886-83dc-4154-b3eb-78163bee4aa3
11cb39e0-4eb3-4884-a574-326c7722ee32
a

In [18]:
async with ss.begin():
    pbar_prod = tqdm(total=product_amount - 1)
    pbar_sub = tqdm()

    current_day = datetime.now()

    for prod in product_ids:
        pbar_prod.set_description(f"Product : {prod}")

        price_amount = random.randint(
            MIN_PRICE_CHANGE_AMOUNT,
            MAX_PRICE_CHANGE_AMOUNT,
        )
        stock_amount = random.randint(
            MIN_STOCK_CHANGE_AMOUNT,
            MAX_STOCK_CHANGE_AMOUNT,
        )

        pbar_sub.reset(price_amount)
        pbar_sub.set_description("Generate price")

        prev_date = current_day
        for _ in range(price_amount):
            price = round(random.uniform(MIN_PRICE, MAX_PRICE), 2)
            sale_percent = round(random.uniform(MIN_SALE, MAX_SALE), 2)

            day_offset = dt.timedelta(
                days=random.randint(
                    MIN_PRICE_CHANGE_INTERVAL_DAY, MAX_PRICE_CHANGE_INTERVAL_DAY
                )
            )

            prev_date -= day_offset

            price_history = ProductPriceHistory(
                product_id=prod,
                date=prev_date,
                price=price,
                sale_percent=sale_percent,
            )

            ss.add(price_history)

            pbar_sub.update()

        pbar_sub.reset(stock_amount)
        pbar_sub.set_description("Generate stock")
        prev_date = current_day
        for _ in range(stock_amount):
            price = round(random.uniform(MIN_PRICE, MAX_PRICE) * 0.7, 2)
            stock = random.randint(
                MIN_STOCK_CHANGE_AMOUNT,
                MAX_STOCK_CHANGE_AMOUNT,
            )
            day_offset = dt.timedelta(
                days=random.randint(MIN_RESTOCK_INTERVAL, MAX_RESTOCK_INTERVAL)
            )

            prev_date -= day_offset
            stock = ProductStockHistory(
                product_id=prod,
                in_stock=stock,
                in_price=price,
                date=prev_date,
            )

            ss.add(stock)

            pbar_sub.update()

        pbar_prod.update()

    await ss.flush()

  0%|          | 0/22 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [22]:
import uuid
from db.postgresql.models.order_history import (
    OrderHistory,
    OrderHistoryItems,
    OrderProcess,
    PaymentStatus,
    OrderStatus,
)
from db.postgresql.models.transaction import PaymentTransaction
from services.shipper import ShippingStatus

pbar = tqdm(total=account_amount - 1, desc="Generate per user")
pbar_sub = tqdm(desc="Order:")
async with ss.begin():
    for i in account_ids:
        order_amount = random.randint(MIN_ORDER_PER_USER, MAX_ORDER_PER_USER)

        current_day = datetime.now()

        prev_date = current_day

        pbar_sub.reset(order_amount - 1)
        for _ in range(order_amount):
            item_for_order = random.randint(
                MIN_ITEM_PER_ORDER,
                MAX_ITEM_PER_ORDER,
            )

            day_offset = dt.timedelta(
                days=random.randint(
                    MIN_ORDER_INTERVAL_DAY,
                    MAX_ORDER_INTERVAL_DAY,
                )
            )

            prev_date -= day_offset

            prods = list(
                (
                    await ss.execute(
                        sqla.select(ProductPriceHistory.product_id)
                        .filter(ProductPriceHistory.date < prev_date)
                        .distinct(ProductPriceHistory.product_id)
                    )
                ).scalars()
            )

            if len(prods) < item_for_order:
                break

            order_items = random.sample(prods, item_for_order)

            total_price = 0

            order_id = uuid.uuid4()

            order_items_db = []
            for item in order_items:
                quantity = random.randint(1, 5)
                prod_price = await ss.scalar(
                    sqla.select(ProductPriceHistory)
                    .filter(
                        ProductPriceHistory.product_id == item,
                        ProductPriceHistory.date < prev_date,
                    )
                    .order_by(ProductPriceHistory.date.desc())
                    .limit(1)
                )

                if not prod_price:
                    raise Exception()

                total_price += (prod_price.price * prod_price.sale_percent) * quantity

                order_items_db.append(
                    OrderHistoryItems(
                        order_history_id=order_id,
                        product_id=prod_price.product_id,
                        date=prod_price.date,
                        quantity=quantity,
                    )
                )

            total_price = round(total_price, 2)
            order_status = (
                random.choices(
                    [
                        OrderStatus.CANCELLED,
                        OrderStatus.ON_CONFIRM,
                        OrderStatus.ON_PROCESSING,
                        OrderStatus.ON_SHIPPING,
                        OrderStatus.SHIPPED,
                        OrderStatus.DELIVERED,
                    ],
                    [
                        0.05,
                        0.05,
                        0.05,
                        0.55,
                        0.2,
                        0.1,
                    ],
                ),
            )[0][0]
            order = OrderHistory(
                id=order_id,
                user_id=i,
                order_date=prev_date,
                delivery_address=fake.address(),
                note=fake.sentence(5),
                total_price=total_price,
                receiver=fake.name(),
                phonenumber=generate_phone(),
                coupon=None,
                payment_method="COD",
                payment_status=PaymentStatus.PENDING,
                order_status=order_status,
            )

            ss.add(order)
            ss.add_all(order_items_db)

            if order_status not in (OrderStatus.ON_CONFIRM, OrderStatus.CANCELLED):
                confirm_date = prev_date + dt.timedelta(hours=random.randint(10, 48))

                shipping_date = confirm_date + dt.timedelta(
                    hours=random.randint(10, 24)
                )

                status = None

                match order_status:
                    case OrderStatus.ON_PROCESSING:
                        shipping_date = None
                        status = random.choice([
                            ShippingStatus.ACCEPTED,
                            ShippingStatus.REJECTED,
                        ])
                    case OrderStatus.ON_SHIPPING:
                        status = ShippingStatus.ON_SHIPPING
                    case OrderStatus.SHIPPED | OrderStatus.DELIVERED:
                        status = ShippingStatus.DELIVERED

                process = OrderProcess(
                    order_id=order_id,
                    confirm_date=confirm_date,
                    process_by=random.choice(staffs),
                    shipping_date=shipping_date,
                    deliver_by=random.choice(shippers),
                    status=status,
                )
                ss.add(process)

            if order_status == OrderStatus.DELIVERED:
                payment = PaymentTransaction(
                    order_id=order.id,
                    create_time=order.order_date,
                    amount=order.total_price,
                    status=PaymentStatus.RECEIVED,
                )
                ss.add(payment)

            pbar_sub.update()
        pbar.update()

    await ss.commit()


Generate per user:   0%|          | 0/99 [00:00<?, ?it/s]

Order:: 0it [00:00, ?it/s]